# Sprawozdanie z laboratorium 6

***Autor: Adam Dąbkowski***

Celem szóstego laboratorium jest zaimplementowanie algorytmu ***Q-learning***. Dodatkowo należy stworzyć agenta rozwiązującego problem ***Taxi***.


## 0. Importowanie niezbędnych bibliotek

In [64]:
import gym
import numpy as np
import pandas as pd
import time

## 1. Wizualizacja stanu środowiska

Wykorzystywane przez na środowisko zawiera cztery wyznaczone miejsca (***R***, ***G***, ***Y***, ***B***), w których pasażer może wsiąść do taksówki (***żółty prostokąt***) lub wysiąść. Gracz otrzymuje pozytywne nagrody za udane podrzucenie pasażera w odpowiednim miejscu, natomiast negatywne nagrody za próby odebrania/odwiezienia pasażera kończące się niepowodzeniem oraz za każdy krok, w którym nie otrzymano kolejnej nagrody.

In [65]:
env = gym.make('Taxi-v3')
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## 2. Implementacja algorytmu ***Q-learning***

Głównym zadaniem szóstego laboratorium jest implementacja algorytmu ***Q-learning***. Po za tym należy stworzyć agenta rozwiązującego problem ***Taxi***. W tym celu stworzona została klasa ***QlearningAgent***. Podczas tworzenia obiektu tej klasy istnieje możliwość podania parametrów ***env*** (*wykorzystywane środowisko*), ***beta*** (*współczynnik uczenia*), ***gamma*** (*stopa dyskontowa*) oraz ***epsilon*** (*prawdopodobieństwo $\epsilon$*).



Klasa ***QlearningAgent*** zawiera także cztery metody:
- ***get_parameters()*** - metoda zwracająca wartości parametrów ***beta***, ***gamma*** oraz ***epsilon***
- ***exploration()*** - metoda odpowiadająca za strategię eksploracji (w tym przypadku ***strategię $\epsilon$-zachłanną***)
- ***learn()*** - metoda odpowiadająca za uczenie według algorytmu ***Q-learning***
- ***evaluate()*** - metoda odpowiedzialna za ocenę na danym etapie uczenia

In [66]:
class QlearningAgent:
    def __init__(self, env, beta=0.03, gamma=0.9, epsilon=0.01):
        self.env = env
        self.beta = beta
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q = np.zeros([env.observation_space.n, env.action_space.n])

    def get_parameters(self):
        return {
            "beta": self.beta,
            "gamma": self.gamma,
            "epsilon": self.epsilon
        }

    def exploration(self, state):
        if np.random.rand() < self.epsilon:
            action = self.env.action_space.sample()
        else:
            action = np.argmax(self.Q[state])
        return action

    def learn(self, n_episodes=20000, n_eval_episodes=500, eval_period=2000, deep_printing=False):
        for i in range(n_episodes):
            state = self.env.reset()
            done = False
            while not done:
                action = self.exploration(state)
                new_state, reward, done, _ = self.env.step(action)
                self.Q[state, action] += self.beta * (reward + self.gamma * np.max(self.Q[new_state, :]) - self.Q[state, action])
                state = new_state


            if (i+1) % eval_period == 0 or (i+1) == n_episodes:
                average_reward = self.evaluate(n_eval_episodes, deep_printing)
                print(f'After {i+1}/{n_episodes} learning episodes - average reward: {average_reward}')
                if deep_printing:
                    print(" ")

        return average_reward


    def evaluate(self, n_eval_episodes, printing=False):
        all_rewards = []
        for i in range(n_eval_episodes):
            episode_reward = 0
            state = self.env.reset()
            done = False
            while not done:
                action = np.argmax(self.Q[state])
                state, reward, done, _ = self.env.step(action)
                episode_reward += reward

            all_rewards.append(episode_reward)

            if printing:
                print(f'Episode {i} reward: {episode_reward}')

        return np.mean(all_rewards)

Aby móc w łatwy sposób prezentować i analizować rezultaty działania algorytmu dla poszczególnych przypadków, zaimplementowana została prosta klasa ***Results***.

In [67]:
class Results:
    def __init__(self):
        self.results = pd.DataFrame(columns=["Learning episodes", "beta", "gamma", "epsilon", "Average reward", "Training time"])

    def update_results(self, n_episodes, beta, gamma, epsilon, average_reward, training_time):
        self.results.loc[len(self.results)] = [n_episodes, beta, gamma, epsilon, average_reward, training_time]

    def delete_row(self, index):
        self.results.drop([index], axis=0, inplace=True)

    def sort_results(self, column_name):
        self.results = self.results.sort_values(by=[column_name])

    def __repr__(self):
        return self.results.to_string()

## 3. Zastosowanie algorytmu

 Mając zaimplementowaną klasę ***QlearningAgent*** oraz funkcje analizujące otrzymywane wyniki, możemy przejść do przeprowadzenia szeregu doświadczeń, dzięki którym zbadamy wpływ poszczególnych parametrów. Na początku ustalmy liczbę epizodów całego procesu uczenia (***n_episodes***), liczbę epizodów pojedynczej ewaluacji (***n_eval_episodes***) oraz okres, który wskazuje na częstotliwość wykonywania ewaluacji (***eval_period***). Są to wartości równe odpowiednio **20000**, **500** oraz **2000**. Poprzez zastosowanie tak dużej liczby epizodów ewaluacyjnych jesteśmy w stanie przedstawić bardziej uśrednione wyniki, a co za tym idzie, precyzyjniejsze wnioski.

In [68]:
n_episodes = 20000
n_eval_episodes = 500
eval_period = 2000

#### 3.1 Badanie wpływu współczynnika $\beta$

Nasze rozważania rozpoczynamy od sprawdzenia działania algorytmu dla wartości domyślnym. W naszym przypadku są ***beta = 0,03***, ***gamma = 0,9*** oraz ***epsilon = 0,01***.

In [69]:
results_beta = Results()

In [70]:
beta = 0.03
gamma = 0.9
epsilon = 0.01

In [71]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [72]:
agent.get_parameters()

{'beta': 0.03, 'gamma': 0.9, 'epsilon': 0.01}

In [73]:
timer_start = time.time()

In [74]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -241.13
After 4000/20000 learning episodes - average reward: -76.094
After 6000/20000 learning episodes - average reward: -25.552
After 8000/20000 learning episodes - average reward: -0.698
After 10000/20000 learning episodes - average reward: 3.342
After 12000/20000 learning episodes - average reward: 7.464
After 14000/20000 learning episodes - average reward: 6.412
After 16000/20000 learning episodes - average reward: 7.932
After 18000/20000 learning episodes - average reward: 8.01
After 20000/20000 learning episodes - average reward: 7.904


In [75]:
results_beta.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

Jak widać, powyżej zadowalające rezultaty otrzymujemy po ok. **12000** epizodach treningowych. Sprawdźmy teraz, co się stanie, gdy wartość współczynnika uczenia $\beta$ zostanie zwiększona do wartości **0,05**, czy algorytm będzie w stanie wcześniej osiągnąć satysfakcjonujące nas wyniki.

In [76]:
beta = 0.05
gamma = 0.9
epsilon = 0.01

In [77]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [78]:
timer_start = time.time()

In [79]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -160.512
After 4000/20000 learning episodes - average reward: -50.546
After 6000/20000 learning episodes - average reward: 1.396
After 8000/20000 learning episodes - average reward: 7.712
After 10000/20000 learning episodes - average reward: 8.118
After 12000/20000 learning episodes - average reward: 7.826
After 14000/20000 learning episodes - average reward: 7.772
After 16000/20000 learning episodes - average reward: 8.02
After 18000/20000 learning episodes - average reward: 8.05
After 20000/20000 learning episodes - average reward: 7.888


In [80]:
results_beta.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

Zgodnie z naszymi przewidywaniami, algorytm znacznie szybciej przekroczył próg o wartości **7**, ponieważ wynik ten został osiągnięty już po ok. **8000**. Chcąc otrzymać podobne rezultaty w jeszcze krótszym czasie, sprawdźmy, co się stanie, gdy współczynnik uczenia zostanie zwiększony do wartości **0,1**.

In [187]:
beta = 0.1
gamma = 0.9
epsilon = 0.01

In [82]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [83]:
timer_start = time.time()

In [84]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -30.722
After 4000/20000 learning episodes - average reward: 7.11
After 6000/20000 learning episodes - average reward: 7.892
After 8000/20000 learning episodes - average reward: 7.922
After 10000/20000 learning episodes - average reward: 8.234
After 12000/20000 learning episodes - average reward: 7.932
After 14000/20000 learning episodes - average reward: 7.876
After 16000/20000 learning episodes - average reward: 7.87
After 18000/20000 learning episodes - average reward: 7.94
After 20000/20000 learning episodes - average reward: 7.818


In [85]:
results_beta.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

Ponownie czas, po którym otrzymujemy świetne wyniki, został skrócony, tym razem do ok. **4000** epizodów. Oczywiście możemy domyślać się, że próg równy **7** może zostać przekroczony znacznie wcześniej, gdyż przyjęty kwant czasu (w naszym przypadku liczba epizodów uczących pomiędzy ewaluacjami) uniemożliwia dostrzeżenie wspomnianej obserwacji. Aby to udowodnić, poniżej zamieszczono wyniki doświadczenia dla mniejszej wartości ***eval_period*** równej **500**.

In [188]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [189]:
timer_start = time.time()

In [190]:
reward = agent.learn(n_episodes=6000, n_eval_episodes=500, eval_period=500)

After 500/6000 learning episodes - average reward: -235.548
After 1000/6000 learning episodes - average reward: -90.672
After 1500/6000 learning episodes - average reward: -48.608
After 2000/6000 learning episodes - average reward: -19.072
After 2500/6000 learning episodes - average reward: 3.72
After 3000/6000 learning episodes - average reward: 6.244
After 3500/6000 learning episodes - average reward: 8.052
After 4000/6000 learning episodes - average reward: 7.79
After 4500/6000 learning episodes - average reward: 7.986
After 5000/6000 learning episodes - average reward: 7.95
After 5500/6000 learning episodes - average reward: 7.792
After 6000/6000 learning episodes - average reward: 7.862


Na koniec sprawdźmy, czy zmniejszenie współczynnika uczenia rzeczywiście poskutkuje znacznie wolniejszym przebiegiem uczenia.

In [89]:
beta = 0.001
gamma = 0.9
epsilon = 0.01

In [90]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [91]:
timer_start = time.time()

In [92]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -232.364
After 4000/20000 learning episodes - average reward: -289.424
After 6000/20000 learning episodes - average reward: -364.754
After 8000/20000 learning episodes - average reward: -268.364
After 10000/20000 learning episodes - average reward: -261.002
After 12000/20000 learning episodes - average reward: -342.524
After 14000/20000 learning episodes - average reward: -332.534
After 16000/20000 learning episodes - average reward: -353.504
After 18000/20000 learning episodes - average reward: -331.8
After 20000/20000 learning episodes - average reward: -335.238


In [93]:
results_beta.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

Jak widać, nasze przypuszczenia ponownie się potwierdziły. Dla ***beta*** równej **0,001** algorytm nie jest w stanie nawet osiągnąć wartości dodatniej.

Poniżej zamieszona jest zbiorcza tabela zawierająca wyniki dotychczasowych doświadczeń.

In [94]:
results_beta.sort_results("beta")

In [95]:
results_beta.results

,Learning episodes,beta,gamma,epsilon,Average reward,Training time
3,20000.0,0.001,0.9,0.01,-335.238,96.062129
0,20000.0,0.030,0.9,0.01,7.904,16.596551
1,20000.0,0.050,0.9,0.01,7.888,12.511721
2,20000.0,0.100,0.9,0.01,7.818,11.106120


Jak widać powyżej, tabela oprócz wartości parametrów opisanych powyżej zawiera dodatkowo kolumnę, w której zamieszczono czas uczenia (włącznie z ewaluacją) dla poszczególnych zestawów parametrów. Łatwo zauważyć, że wraz ze wzrostem współczynnika $\beta$ odnotowujemy krótszy czas potrzebny na osiągnięcie końcowego wyniku. Zdecydowanie najgorzej pod tym względem wypada zestaw z najmniejszą wartością współczynnika ***beta***.

#### 3.2 Badanie wpływu współczynnika $\gamma$

Kolejnym z badanych parametrów jest stopa dyskontowa **$\gamma$**. Jeśli jej wartość będzie bliska **0**, wówczas przyszłe nagrody będą miały marginalne znaczenie w stosunku do bieżących nagród, natomiast jeśli **$\gamma$** będzie przyjmować wartości zbliżające się do **1**, to przyszłe nagrody będą miały niemal identyczne znaczenie co nagrody bieżące.

Na początku sprawdźmy, czy zwiększenie stopy dyskontowej będzie pozytywnie wpływało na szybkość uczenia. W tym celu przeprowadzimy trzy doświadczenia, w których $\gamma$ równa będzie **0,95**, **0,99** oraz **0,999**.

In [96]:
results_gamma = Results()

In [97]:
beta = 0.03
gamma = 0.95
epsilon = 0.01

In [98]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [99]:
timer_start = time.time()

In [100]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -249.972
After 4000/20000 learning episodes - average reward: -86.492
After 6000/20000 learning episodes - average reward: -23.058
After 8000/20000 learning episodes - average reward: 0.482
After 10000/20000 learning episodes - average reward: 7.73
After 12000/20000 learning episodes - average reward: 7.864
After 14000/20000 learning episodes - average reward: 7.966
After 16000/20000 learning episodes - average reward: 8.202
After 18000/20000 learning episodes - average reward: 7.76
After 20000/20000 learning episodes - average reward: 7.942


In [101]:
results_gamma.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

In [102]:
beta = 0.03
gamma = 0.99
epsilon = 0.01

In [103]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [104]:
timer_start = time.time()

In [105]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -239.664
After 4000/20000 learning episodes - average reward: -87.298
After 6000/20000 learning episodes - average reward: -51.04
After 8000/20000 learning episodes - average reward: -18.342
After 10000/20000 learning episodes - average reward: 7.394
After 12000/20000 learning episodes - average reward: 7.742
After 14000/20000 learning episodes - average reward: 7.984
After 16000/20000 learning episodes - average reward: 7.908
After 18000/20000 learning episodes - average reward: 7.794
After 20000/20000 learning episodes - average reward: 7.938


In [106]:
results_gamma.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

In [107]:
beta = 0.03
gamma = 0.999
epsilon = 0.01

In [108]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [109]:
timer_start = time.time()

In [110]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -322.798
After 4000/20000 learning episodes - average reward: -98.87
After 6000/20000 learning episodes - average reward: -15.416
After 8000/20000 learning episodes - average reward: 5.058
After 10000/20000 learning episodes - average reward: 8.076
After 12000/20000 learning episodes - average reward: 7.728
After 14000/20000 learning episodes - average reward: 7.714
After 16000/20000 learning episodes - average reward: 7.984
After 18000/20000 learning episodes - average reward: 8.134
After 20000/20000 learning episodes - average reward: 7.886


In [111]:
results_gamma.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

O ile nie dostrzegamy poważnych różnic pomiędzy wynikami dla trzech powyższych symulacji, o tyle łatwo zauważyć poprawę względem wartości, które przyjęliśmy za domyślne. Sprawdźmy zatem, czy zmniejszenie współczynnika ***gamma*** względem wartości początkowej, będzie negatywne w skutkach.

In [112]:
beta = 0.03
gamma = 0.8
epsilon = 0.01

In [113]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [114]:
timer_start = time.time()

In [115]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -309.412
After 4000/20000 learning episodes - average reward: -135.292
After 6000/20000 learning episodes - average reward: -86.72
After 8000/20000 learning episodes - average reward: -30.642
After 10000/20000 learning episodes - average reward: -19.504
After 12000/20000 learning episodes - average reward: -11.708
After 14000/20000 learning episodes - average reward: -7.332
After 16000/20000 learning episodes - average reward: -3.884
After 18000/20000 learning episodes - average reward: 1.5
After 20000/20000 learning episodes - average reward: -3.672


In [116]:
results_gamma.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

In [117]:
beta = 0.03
gamma = 0.6
epsilon = 0.01

In [118]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [119]:
timer_start = time.time()

In [120]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -218.396
After 4000/20000 learning episodes - average reward: -149.81
After 6000/20000 learning episodes - average reward: -110.17
After 8000/20000 learning episodes - average reward: -106.014
After 10000/20000 learning episodes - average reward: -95.364
After 12000/20000 learning episodes - average reward: -81.576
After 14000/20000 learning episodes - average reward: -57.154
After 16000/20000 learning episodes - average reward: -45.768
After 18000/20000 learning episodes - average reward: -46.934
After 20000/20000 learning episodes - average reward: -40.864


In [121]:
results_gamma.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

Zarówno w jednym, jak i drugim przypadku wyniki są zauważalnie gorsze, gdyż nie jesteśmy w stanie uzyskać dodatniego finalnego wyniku dla danej liczby epizodów procesu uczenia.

In [122]:
results_gamma.sort_results("gamma")

In [123]:
results_gamma.results

,Learning episodes,beta,gamma,epsilon,Average reward,Training time
4,20000.0,0.03,0.600,0.01,-40.864,22.380610
3,20000.0,0.03,0.800,0.01,-3.672,16.822754
0,20000.0,0.03,0.950,0.01,7.942,14.996722
1,20000.0,0.03,0.990,0.01,7.938,15.066344
2,20000.0,0.03,0.999,0.01,7.886,13.992574


W tym przypadku również jesteśmy w stanie dostrzec pewną prawidłowość dotyczącą czasu trwania uczenia. Tym razem to wzrost stopy dyskontowej powoduje skrócenie potrzebnego czasu.

#### 3.3 Badanie wpływu wartości parametru $\epsilon$

Mając na uwadze fakt, że przyjętą przez nas strategią eksploracji jest strategia $\epsilon$ - zachłanna, istotnym parametrem jest $\epsilon$ - prawdopodobieństwo wyboru losowego, przy prawdopodobieństwie wyboru czynności o największej Q-wartości równym 1-$\epsilon$.

Chcąc sprawdzić, czy zwiększenie eksploracji poprzez zwiększenie losowości pozytywnie wpłynie na czas otrzymywania satysfakcjonujących wyników, przeprowadzone zostaną doświadczenia o parametrze ***epsilon*** równym kolejno **0,05**, **0,1**, **0,2**, **0,5** oraz **1**.

In [124]:
results_epsilon = Results()

In [125]:
beta = 0.03
gamma = 0.9
epsilon = 0.05

In [126]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [127]:
timer_start = time.time()

In [128]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -269.134
After 4000/20000 learning episodes - average reward: -146.474
After 6000/20000 learning episodes - average reward: -23.794
After 8000/20000 learning episodes - average reward: -0.398
After 10000/20000 learning episodes - average reward: -1.988
After 12000/20000 learning episodes - average reward: 6.326
After 14000/20000 learning episodes - average reward: 7.756
After 16000/20000 learning episodes - average reward: 8.012
After 18000/20000 learning episodes - average reward: 7.872
After 20000/20000 learning episodes - average reward: 7.768


In [129]:
results_epsilon.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

In [130]:
beta = 0.03
gamma = 0.9
epsilon = 0.1

In [131]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [132]:
timer_start = time.time()

In [133]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -167.234
After 4000/20000 learning episodes - average reward: -94.964
After 6000/20000 learning episodes - average reward: -24.282
After 8000/20000 learning episodes - average reward: -7.204
After 10000/20000 learning episodes - average reward: 2.556
After 12000/20000 learning episodes - average reward: 5.042
After 14000/20000 learning episodes - average reward: 8.062
After 16000/20000 learning episodes - average reward: 7.89
After 18000/20000 learning episodes - average reward: 7.986
After 20000/20000 learning episodes - average reward: 7.948


In [134]:
results_epsilon.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

In [135]:
beta = 0.03
gamma = 0.9
epsilon = 0.2

In [136]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [137]:
timer_start = time.time()

In [138]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -157.27
After 4000/20000 learning episodes - average reward: -83.16
After 6000/20000 learning episodes - average reward: -24.73
After 8000/20000 learning episodes - average reward: -7.616
After 10000/20000 learning episodes - average reward: 3.09
After 12000/20000 learning episodes - average reward: 6.996
After 14000/20000 learning episodes - average reward: 7.698
After 16000/20000 learning episodes - average reward: 8.15
After 18000/20000 learning episodes - average reward: 7.892
After 20000/20000 learning episodes - average reward: 7.912


In [139]:
results_epsilon.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

In [140]:
beta = 0.03
gamma = 0.9
epsilon = 0.5

In [141]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [142]:
timer_start = time.time()

In [143]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -166.172
After 4000/20000 learning episodes - average reward: -73.556
After 6000/20000 learning episodes - average reward: -13.212
After 8000/20000 learning episodes - average reward: -2.132
After 10000/20000 learning episodes - average reward: 4.328
After 12000/20000 learning episodes - average reward: 7.928
After 14000/20000 learning episodes - average reward: 7.872
After 16000/20000 learning episodes - average reward: 8.006
After 18000/20000 learning episodes - average reward: 7.952
After 20000/20000 learning episodes - average reward: 7.938


In [144]:
results_epsilon.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

In [145]:
beta = 0.03
gamma = 0.9
epsilon = 1

In [146]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [147]:
timer_start = time.time()

In [148]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -166.674
After 4000/20000 learning episodes - average reward: 7.266
After 6000/20000 learning episodes - average reward: 7.664
After 8000/20000 learning episodes - average reward: 8.094
After 10000/20000 learning episodes - average reward: 8.01
After 12000/20000 learning episodes - average reward: 8.1
After 14000/20000 learning episodes - average reward: 7.972
After 16000/20000 learning episodes - average reward: 7.702
After 18000/20000 learning episodes - average reward: 8.09
After 20000/20000 learning episodes - average reward: 7.914


In [149]:
results_epsilon.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

Otrzymane wyniki w większości nie odbiegają znacząco od tych, które otrzymaliśmy dla wartości początkowych, ponieważ zadowalające wartości otrzymujemy w przybliżeniu po ok. **12000 - 14000** epizodach uczenia. Ciekawe rezultaty otrzymaliśmy natomiast dla ***epsilona*** równego 1, wskazującego na całkowicie losowe podejście do strategi eksploracji. O dziwo wyniki są rewelacyjne. Jednakże wynika to głównie ze szczęśliwego trafu i specyfiki problemu.

Wiedząc, jaki wpływ ma duża wartość parametru ***epsilon***, sprawdźmy, co się stanie, gdy losowość podczas eksploracji zostanie zmniejszona. W tym celu przeprowadzone zostaną doświadczenia dla $\epsilon$ równego **0,005**, **0,001**, **0,0001** oraz **0**.

In [150]:
beta = 0.03
gamma = 0.9
epsilon = 0.005

In [151]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [152]:
timer_start = time.time()

In [153]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -172.674
After 4000/20000 learning episodes - average reward: -88.818
After 6000/20000 learning episodes - average reward: -26.568
After 8000/20000 learning episodes - average reward: -2.516
After 10000/20000 learning episodes - average reward: 5.088
After 12000/20000 learning episodes - average reward: 6.35
After 14000/20000 learning episodes - average reward: 7.882
After 16000/20000 learning episodes - average reward: 8.08
After 18000/20000 learning episodes - average reward: 7.844
After 20000/20000 learning episodes - average reward: 7.914


In [154]:
results_epsilon.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

In [155]:
beta = 0.03
gamma = 0.9
epsilon = 0.001

In [156]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [157]:
timer_start = time.time()

In [158]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -227.332
After 4000/20000 learning episodes - average reward: -97.596
After 6000/20000 learning episodes - average reward: -25.944
After 8000/20000 learning episodes - average reward: -13.264
After 10000/20000 learning episodes - average reward: -13.658
After 12000/20000 learning episodes - average reward: 5.814
After 14000/20000 learning episodes - average reward: 7.948
After 16000/20000 learning episodes - average reward: 7.906
After 18000/20000 learning episodes - average reward: 7.908
After 20000/20000 learning episodes - average reward: 7.874


In [159]:
results_epsilon.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

In [160]:
beta = 0.03
gamma = 0.9
epsilon = 0.0001

In [161]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [162]:
timer_start = time.time()

In [163]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -212.84
After 4000/20000 learning episodes - average reward: -141.592
After 6000/20000 learning episodes - average reward: -30.74
After 8000/20000 learning episodes - average reward: -4.714
After 10000/20000 learning episodes - average reward: 2.566
After 12000/20000 learning episodes - average reward: 7.824
After 14000/20000 learning episodes - average reward: 7.766
After 16000/20000 learning episodes - average reward: 7.836
After 18000/20000 learning episodes - average reward: 8.096
After 20000/20000 learning episodes - average reward: 7.992


In [164]:
results_epsilon.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

In [165]:
beta = 0.03
gamma = 0.9
epsilon = 0

In [166]:
agent = QlearningAgent(env=env, beta=beta, gamma=gamma, epsilon=epsilon)

In [167]:
timer_start = time.time()

In [168]:
reward = agent.learn(n_episodes=n_episodes, n_eval_episodes=n_eval_episodes, eval_period=eval_period)

After 2000/20000 learning episodes - average reward: -188.192
After 4000/20000 learning episodes - average reward: -88.032
After 6000/20000 learning episodes - average reward: -54.956
After 8000/20000 learning episodes - average reward: -12.762
After 10000/20000 learning episodes - average reward: 5.016
After 12000/20000 learning episodes - average reward: 7.524
After 14000/20000 learning episodes - average reward: 7.394
After 16000/20000 learning episodes - average reward: 7.954
After 18000/20000 learning episodes - average reward: 7.93
After 20000/20000 learning episodes - average reward: 7.772


In [169]:
results_epsilon.update_results(n_episodes, beta, gamma, epsilon, reward, time.time() - timer_start)

Otrzymane rezultaty przypominają te, które obserwowaliśmy dla większych wartości ***epsilon***. Dzięki temu wiemy, żeby wpływ parametru $\epsilon$ nie jest tak duży, jak ma to miejsce dla współczynnika uczenia czy stopy dyskontowej.

Poniżej, analogicznie jak wcześniej, zamieszczona została tabela zawierająca wyniki przeprowadzone w omawianym podpunkcie.

In [170]:
results_epsilon.sort_results("epsilon")

In [171]:
results_epsilon.results

,Learning episodes,beta,gamma,epsilon,Average reward,Training time
8,20000.0,0.03,0.9,0.0000,7.772,14.990079
7,20000.0,0.03,0.9,0.0001,7.992,15.041967
6,20000.0,0.03,0.9,0.0010,7.874,14.859613
5,20000.0,0.03,0.9,0.0050,7.914,15.160997
0,20000.0,0.03,0.9,0.0500,7.768,15.504930
1,20000.0,0.03,0.9,0.1000,7.948,16.062076
2,20000.0,0.03,0.9,0.2000,7.912,17.421455
3,20000.0,0.03,0.9,0.5000,7.938,24.528253
4,20000.0,0.03,0.9,1.0000,7.914,109.131963


Analizując powyższą tabelę, dostrzegamy jednak, że przyjmowanie dużych wartości $\epsilon$ nie pozostaje bez echa. Z im większą jego wartością mamy do czynienia, tym wyraźniej wzrasta czas potrzebny na uzyskanie końcowego wyniku, co szczególnie widać dla ***epsilon*** równego **0,5** i **1**.

## 4. Podsumowanie

 Dla zwiększenia czytelności otrzymanych rezultatów, poniżej została zamieszczona zbiorcza tabela, która przedstawia wyniki powyższych doświadczeń.

In [172]:
results = pd.concat([results_beta.results, results_gamma.results, results_epsilon.results])

In [173]:
results

,Learning episodes,beta,gamma,epsilon,Average reward,Training time
3,20000.0,0.001,0.900,0.0100,-335.238,96.062129
0,20000.0,0.030,0.900,0.0100,7.904,16.596551
1,20000.0,0.050,0.900,0.0100,7.888,12.511721
2,20000.0,0.100,0.900,0.0100,7.818,11.106120
4,20000.0,0.030,0.600,0.0100,-40.864,22.380610
3,20000.0,0.030,0.800,0.0100,-3.672,16.822754
0,20000.0,0.030,0.950,0.0100,7.942,14.996722
1,20000.0,0.030,0.990,0.0100,7.938,15.066344
2,20000.0,0.030,0.999,0.0100,7.886,13.992574
8,20000.0,0.030,0.900,0.0000,7.772,14.990079


Mając na uwadze dotychczasowe obserwacje, a także wyniki zamieszczone w powyższej tabeli, możemy dojść do szeregu wniosków. Pierwszym z nich jest fakt, że wraz ze wzrostem współczynnika uczenia $\beta$ jesteśmy w stanie znacząco zredukować liczbę epizodów koniecznych do osiągnięcia satysfakcjonujących nas rezultatów. W przypadku jego zbyt małej wartości może zdarzyć się, że już samo osiągnięcie dodatniego wyniku będzie wynikiem niemożliwym. Podobną sytuację możemy zaobserwować przypadku stopy dyskontowej $\gamma$. Tutaj również w przypadku zbyt małej wartości nie jesteśmy w stanie osiągnąć dodatniego wyniku, nie mówiąc już o przekroczeniu progu, z którego możemy być zadowoleni.

Inaczej wygląda sytuacja z parametrem $\epsilon$. W tym przypadku, na podstawie otrzymanych rezultatów, nie zauważamy istotnego wpływu wartości wspomnianego parametru na jakość uczenia, w tym wymaganą liczbę epizodów treningowych. Warto jednak wspomnieć, że wraz ze wzrostem wartości parametru ***epsilon*** obserwujemy dłuższy czas potrzebny na ukończenie pełnego procesu uczenia. Oczywiście w wyniku doświadczenia, w którym $\epsilon$ był równy **1**, otrzymujemy rewelacyjne rezultaty już po ok. **4000** epizodów, jednakże otrzymany wynik należy skonfrontować z czasem, który był potrzebny na wykonanie **20000** epizodów. Czas ten jest zdecydowanie najdłuższy. W przypadku pozostałych parametrów również obserwujemy pewne zależności związane z czasem wykonania, jednakże w przeciwieństwie do parametru $\epsilon$, na zwiększenie czasu uczenia wpływają coraz mniejsze wartości parametrów $\beta$ i $\gamma$.

Na potrzeby zestawienia ze sobą rezultatów dla różnych zestawów parametrów, w celu zbadania wpływu poszczególnego parametru i uzyskania miarodajnego wyniku, przyjęto tę samą liczbę epizodów uczących i ewaluacyjnych w trakcie jednego cyklu. Jednakże przy finalnym doborze parametrów, należy wziąć pod uwagę to, że w większości przypadków satysfakcjonujący nas wynik jesteśmy w stanie uzyskać po znacznie mniejszej ilości epizodów treningowych, niż **20000**. Wówczas powinniśmy wziąć poprawkę, że rzeczywisty czas potrzebny na osiągnięcie takiego wyniku jest znacznie krótszy. W związku z tym możemy stwierdzić, że najlepszym z rozważanych przez nas zestawów parametrów jest ten gdzie $\beta$ jest równa **0,1**, $\gamma$ wynosi **0,9**, natomiast $\epsilon$ przyjmuje wartość **0,01**. Uzyskujemy wówczas czas uczenia wynoszący **11,106 s**, jednakże mając na uwadze fakt, że dobry wynik uzyskaliśmy już po ok. **4000** epizodów treningowych, możemy założyć, że zużycie zasobów, jakim niewątpliwie jest czas, również zostanie zredukowane.
